In [1]:
from astropy.io.fits import getdata, getheader
import astropy.units as u
from nustar_lunar_pointing.tracking import get_epoch_tle
from nustar_lunar_pointing.tracking import convert_nustar_time
from nustar_lunar_pointing.tracking import get_moon_j2000

import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
from datetime import datetime

checktime = datetime.strptime('2017-05-08', "%Y-%m-%d")
tlefile = '../data/NuSTAR.tle'
mindt, line1, line2 = get_epoch_tle(checktime, tlefile)
print('Days between TLE entry and when you want to observe: ', mindt)

Days between TLE entry and when you want to observe:  15


In [39]:
from astropy.coordinates import SkyCoord
from datetime import timedelta
ra = []
dec =[]
times= []
base_ra = None
base_dec = None

step_size = timedelta(0, 10.) # 2 second steps for checking the Moon motion

# Start and stop time for the unocculted period that we're looking at:
checktime = datetime.strptime('2017-05-08T10:00:00', "%Y-%m-%dT%H:%M:%S")
end_time = datetime.strptime('2017-05-08T13:00:00', "%Y-%m-%dT%H:%M:%S")

last_pos = None
dt = []
dra = []
ddec= []
times = []

# How large a motion of the moon you can have:
limit = 2 #arcminutes

set = 0
ctr = 0
last_time = None
base_ra = None

while ( (end_time - checktime).total_seconds() ) > 0:
    checktime += step_size
    
    ra_moon, dec_moon = get_moon_j2000(checktime, line1, line2)
    this_pos = SkyCoord(ra_moon, dec_moon)
    
    # Init values:
    if base_ra is None:
        base_ra = ra_moon
        base_dec = dec_moon
    else:
        dra.extend([ra_moon - base_ra])
        ddec.extend([dec_moon - base_dec])

        
    # Check to see if you're updating the interval or not:
    if set == 0:
        last_pos = this_pos
        
        if last_time is not None:
            dt.extend([checktime - last_time])
            times.extend([checktime])
            ctr += 1

        last_time = checktime
        set = 1
        
    else:
        
        if( this_pos.separation(last_pos).arcmin > limit):
            set = 0

In [43]:
ctr = 0
exp = 0.
all_exp = 0.
slew_rate = 0.17 # arcmin / sec
slew_time = limit / slew_rate
slew_time = 0. #seconds for a 2 arcminute slew

for ind, t in enumerate(dt):
    if (t.total_seconds() - slew_time > 0):
        ctr += 1
        exp += t.total_seconds() - slew_time
        print(times[ind], 'Dwell time: ', t.total_seconds() - slew_time)
#        print(t.tal_seconds())
    all_exp += t.total_seconds()
print()
print('Stationary exposure: ', exp)
print('Total Exposure: ', all_exp)
print('Efficiency: ', exp / all_exp)
print('Number of dwells: ', ctr, '(Used)', len(dt), '(Total)')

IndexError: list index out of range

In [23]:
import astropy.units as u
import numpy as np

In [31]:
alt = 600 * u.km
x = u.R_earth.to(u.km)*u.km + alt
moon_peri = 362.6e3 * u.km
moon_apo = 405.4e3 * u.km

par_peri = np.arctan(x / moon_peri)
par_apo = np.arctan(x / moon_apo)

print(2*par_peri.to(u.arcmin), 2*par_apo.to(u.arcmin))

132.300626771158 arcmin 118.33594332459083 arcmin


In [20]:
u.R_earth.to(u.km) * u.km

<Quantity 6378.136 km>

In [44]:
times

[]